In [2]:
from PIL import Image
from zipfile import ZipFile
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelBinarizer

import os
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

# Loading Images from Disk

In [2]:
!rm -rf ../data/train_data; rm ../data/train_data

rm: cannot remove '../data/train_data': No such file or directory


In [3]:
data_path = Path('../data/')
data_zips = [data_path / f for f in os.listdir(data_path)]

for i, f in enumerate(data_zips):
    print(f'{i}: {f}')

0: ../data/train.zip
1: ../data/test.zip


In [4]:
f_index = 0
data_file = data_zips[f_index]
print(data_file)

zip_folder_name = 'train_data'
data_extract_location = data_path / zip_folder_name

../data/train.zip


In [6]:
with ZipFile(data_file) as z:
    z.extractall(data_extract_location)

In [7]:
inner_folder = os.listdir(data_extract_location)[0];
data_folder_unzip = data_extract_location / inner_folder
os.listdir(data_folder_unzip)

['labels.csv', 'images']

In [8]:
labels = pd.read_csv(data_folder_unzip / 'labels.csv')
labels

,timestamp,forward/backward,left/right
0,1637291664326,0,0
1,1637291664353,0,0
2,1637291664404,0,0
3,1637291664547,0,0
4,1637291664573,0,0
...,...,...,...
17803,1637292329140,0,0
17804,1637292329188,0,0
17805,1637292329210,0,0
17806,1637292329259,0,0


In [11]:
label_encoded = labels['left/right'].values
encoder = LabelBinarizer()
encoder.fit([-1, 0, 1])
label_encoded = encoder.transform(label_encoded)
print("-1: ", label_encoded[labels['left/right'] == -1][0])
print(" 0: ", label_encoded[labels['left/right'] == 0][0])
print("1: ", label_encoded[labels['left/right'] == 1][0])

-1:  [1 0 0]
 0:  [0 1 0]
1:  [0 0 1]


In [12]:
# original (320, 240)
image_rescale_size = (80, 60)
batch_size = 32
data_path = data_folder_unzip
val_split = 0.1
print(f'{data_path}')


train_ds = tf.keras.utils.image_dataset_from_directory(
    str(data_path),
    labels=list(label_encoded),
    label_mode="int",
    color_mode="rgb",
    batch_size=batch_size,
    image_size=image_rescale_size,
    shuffle=True,
    validation_split=val_split,
    subset='training',
    seed=42
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    str(data_path),
    labels=list(label_encoded),
    label_mode="int",
    color_mode="rgb",
    batch_size=batch_size,
    image_size=image_rescale_size,
    shuffle=True,
    validation_split=val_split,
    subset='validation',
    seed=42
)

../data/train_data/dataTrackV2_4
Found 17808 files belonging to 1 classes.
Using 16028 files for training.
Found 17808 files belonging to 1 classes.
Using 1780 files for validation.


In [13]:
for i, el in enumerate(train_ds):
    # print(data.shape, labels.shape)
    # print(cur_labels[:2])
    print(el[0].shape, el[1].shape)
    if i == 5:
        break


(32, 80, 60, 3) (32, 3)
(32, 80, 60, 3) (32, 3)
(32, 80, 60, 3) (32, 3)
(32, 80, 60, 3) (32, 3)
(32, 80, 60, 3) (32, 3)
(32, 80, 60, 3) (32, 3)


# Preprocessing 

## Filter labels and images where the car was moving

In [14]:
filtered_index = (labels['forward/backward'] != 0)
np_filter = np.array([i for i, index in enumerate(filtered_index) if index == False])
bad_indices = tf.convert_to_tensor(np_filter); bad_indices.shape

TensorShape([494])

In [15]:
# def func(index, val):
#     val_tensor = tf.equal(index, bad_indices)
#     ans_tensor = tf.reduce_any(tf.cast(val_tensor, tf.bool)) 
#     # print(ans_tensor)
#     return ans_tensor
# train_ds = train_ds.enumerate().filter(func)
# #train_ds = test.map(lambda i, el: el[1])

# val_ds = val_ds.enumerate().filter(func)
# # val_ds = val_ds.map(lambda index, value: value)

In [16]:
# for i, el in enumerate(train_ds):
#     # print(data.shape, labels.shape)
#     # print(cur_labels[:2])
#     print(el[1][0].shape, el[1][1].shape)
#     if i == 5:
#         break

# Dense Neural Network


In [17]:
model1 = Sequential()
model1.add(Flatten())
model1.add(Rescaling(scale=1./255))
model1.add(Dense(32))
model1.add(Dropout(0.2)),
model1.add(Dense(64))
model1.add(Dropout(0.2)),
model1.add(Dense(128))
model1.add(Dropout(0.2)),
model1.add(Dense(64))
model1.add(Dropout(0.2)),
model1.add(Dense(32))
model1.add(Dense(3, activation='softmax'))

In [18]:
model_save_path = f'../models/dense.h5'

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_save_path,
    monitor="val_loss",
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode="auto",
    save_freq="epoch",
)

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', min_delta=1e-6, patience=10, verbose=2,
    mode='min', baseline=0.8, restore_best_weights=False
)

In [19]:
model1.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])
#model.summary()

In [20]:
hist1 = model1.fit(train_ds, validation_data=val_ds, epochs=50, callbacks=[checkpoint])

Epoch 1/50
501/501 [==============================] - 4s 5ms/step - loss: 0.9707 - accuracy: 0.7348 - val_loss: 0.5794 - val_accuracy: 0.7916
Epoch 2/50
501/501 [==============================] - 2s 4ms/step - loss: 0.5566 - accuracy: 0.7926 - val_loss: 0.5526 - val_accuracy: 0.7983
Epoch 3/50
501/501 [==============================] - 2s 5ms/step - loss: 0.5055 - accuracy: 0.8020 - val_loss: 0.4822 - val_accuracy: 0.8039
Epoch 4/50
501/501 [==============================] - 2s 5ms/step - loss: 0.4800 - accuracy: 0.8103 - val_loss: 0.4708 - val_accuracy: 0.8146
Epoch 5/50
501/501 [==============================] - 2s 4ms/step - loss: 0.4559 - accuracy: 0.8165 - val_loss: 0.4036 - val_accuracy: 0.8399
Epoch 6/50
501/501 [==============================] - 2s 4ms/step - loss: 0.4345 - accuracy: 0.8244 - val_loss: 0.4960 - val_accuracy: 0.8056
Epoch 7/50
501/501 [==============================] - 2s 4ms/step - loss: 0.4395 - accuracy: 0.8260 - val_loss: 0.3931 - val_accuracy: 0.8247
Epoch 

# CNN

In [21]:
model2 = Sequential()
model2.add(Rescaling(scale=1./255))
model2.add(Conv2D(32, (3,3)))
model2.add(MaxPool2D(2,2))
model2.add(Conv2D(64, (3,3)))
model2.add(MaxPool2D(2,2))
model2.add(Conv2D(32, (3,3)))
model2.add(Flatten())
model2.add(Dropout(0.2)),
model2.add(Dense(32))
model2.add(Dropout(0.2)),
model2.add(Dense(64))
model2.add(Dense(3, activation='softmax'))

In [22]:
model2.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              metrics=['accuracy'])

es = tf.keras.callbacks.EarlyStopping(
    monitor='accuracy', min_delta=1e-6, patience=10, verbose=2,
    mode='min', baseline=0.8, restore_best_weights=False
)

model_save_path = f'../models/cnn.h5'

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_save_path,
    monitor="val_loss",
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode="auto",
    save_freq="epoch",
)

In [23]:
hist2 = model2.fit(train_ds, validation_data=val_ds, callbacks=[es, checkpoint], epochs=45)

Epoch 1/45
501/501 [==============================] - 12s 5ms/step - loss: 4.7004 - accuracy: 0.7665 - val_loss: 0.4975 - val_accuracy: 0.8017
Epoch 2/45
501/501 [==============================] - 2s 5ms/step - loss: 0.4467 - accuracy: 0.8169 - val_loss: 0.3694 - val_accuracy: 0.8455
Epoch 3/45
501/501 [==============================] - 2s 5ms/step - loss: 0.3834 - accuracy: 0.8336 - val_loss: 0.3435 - val_accuracy: 0.8584
Epoch 4/45
501/501 [==============================] - 2s 5ms/step - loss: 0.3624 - accuracy: 0.8422 - val_loss: 0.3487 - val_accuracy: 0.8472
Epoch 5/45
501/501 [==============================] - 2s 5ms/step - loss: 0.3442 - accuracy: 0.8512 - val_loss: 0.3385 - val_accuracy: 0.8584
Epoch 6/45
501/501 [==============================] - 2s 5ms/step - loss: 0.3305 - accuracy: 0.8587 - val_loss: 0.3413 - val_accuracy: 0.8551
Epoch 7/45
501/501 [==============================] - 2s 5ms/step - loss: 0.3186 - accuracy: 0.8635 - val_loss: 0.3200 - val_accuracy: 0.8618
Epoch

# Testing

## Loading Dataset

In [24]:
!rm -rf ../data/test_data; rm ../data/test_data

rm: cannot remove '../data/test_data': No such file or directory


In [25]:
data_path = Path('../data/')
data_zips = [data_path / f for f in os.listdir(data_path)]

for i, f in enumerate(data_zips):
    print(f'{i}: {f}')

0: ../data/train.zip
1: ../data/test.zip
2: ../data/train_data


In [28]:
f_index = 1
data_file = data_zips[f_index]
print(data_file)

zip_folder_name = 'test_data'
data_extract_location = data_path / zip_folder_name

../data/test.zip


In [29]:
with ZipFile(data_file) as z:
    z.extractall(data_extract_location)

In [30]:
inner_folder = os.listdir(data_extract_location)[0];
data_folder_unzip = data_extract_location / inner_folder
os.listdir(data_folder_unzip)

['labels.csv', 'images']

In [31]:
labels = pd.read_csv(data_folder_unzip / 'labels.csv')
labels

,timestamp,forward/backward,left/right
0,1637292461661,0,0
1,1637292461900,0,0
2,1637292461951,0,0
3,1637292461993,0,0
4,1637292462034,0,0
...,...,...,...
12686,1637292930962,0,0
12687,1637292930986,0,0
12688,1637292931017,0,0
12689,1637292931063,0,0


In [33]:
labels_orig = labels['left/right'].values
encoder = LabelBinarizer()
encoder.fit([-1, 0, 1])
label_encoded = encoder.transform(labels_orig)
print("-1: ", label_encoded[labels['left/right'] == -1][0])
print(" 0: ", label_encoded[labels['left/right'] == 0][0])
print("1: ", label_encoded[labels['left/right'] == 1][0])

-1:  [1 0 0]
 0:  [0 1 0]
1:  [0 0 1]


In [34]:
print(data_folder_unzip)

../data/test_data/dataTrackV2_5


In [35]:
image_rescale_size = (80, 60)
batch_size = 64
data_path = data_folder_unzip
val_split = 0.1
print(f'{data_path}')


test_ds = tf.keras.utils.image_dataset_from_directory(
    str(data_path),
    labels=list(label_encoded),
    label_mode="int",
    color_mode="rgb",
    batch_size=batch_size,
    image_size=image_rescale_size,
    shuffle=False,
)

../data/test_data/dataTrackV2_5
Found 12691 files belonging to 1 classes.


In [36]:
for i, el in enumerate(test_ds):
    # print(data.shape, labels.shape)
    # print(cur_labels[:2])
    print(el[0].shape, el[1].shape)
    if i == 5:
        break


(64, 80, 60, 3) (64, 3)
(64, 80, 60, 3) (64, 3)
(64, 80, 60, 3) (64, 3)
(64, 80, 60, 3) (64, 3)
(64, 80, 60, 3) (64, 3)
(64, 80, 60, 3) (64, 3)


## Load Models

In [37]:
from tensorflow.keras.models import load_model

In [38]:
model_dir = Path('../models')
models = [model_dir / model for model in os.listdir(model_dir)]; models

[PosixPath('../models/cnn.h5'), PosixPath('../models/dense.h5')]

In [39]:
cnn_model = load_model('../models/cnn.h5'); cnn_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_1 (Rescaling)      (None, 80, 60, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 78, 58, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 39, 29, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 37, 27, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 18, 13, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 11, 32)        18464     
_________________________________________________________________
flatten_1 (Flatten)          (None, 5632)             

In [40]:
dense_model = load_model('../models/dense.h5'); dense_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 14400)             0         
_________________________________________________________________
rescaling (Rescaling)        (None, 14400)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                460832    
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8

In [41]:
cnn_pred = cnn_model.predict(test_ds); cnn_pred

array([[1.1877699e-03, 9.9743611e-01, 1.3761341e-03],
       [9.0792158e-04, 9.9755019e-01, 1.5419233e-03],
       [9.0792158e-04, 9.9755019e-01, 1.5419233e-03],
       ...,
       [1.2976663e-01, 8.7010902e-01, 1.2431802e-04],
       [1.2976663e-01, 8.7010902e-01, 1.2431802e-04],
       [1.3719736e-01, 8.6271143e-01, 9.1256225e-05]], dtype=float32)

In [42]:
pred_labels_cnn = np.argmax(cnn_pred, axis=1) - 1; pred_labels_cnn
np.unique(pred_labels_cnn, return_counts=True)

(array([-1,  0,  1]), array([ 1407, 10234,  1050]))

In [43]:
dense_pred = dense_model.predict(test_ds); dense_pred

array([[1.8935418e-05, 9.9904770e-01, 9.3329744e-04],
       [1.8375000e-05, 9.9913883e-01, 8.4278913e-04],
       [1.8375000e-05, 9.9913883e-01, 8.4278913e-04],
       ...,
       [9.2368059e-02, 9.0233803e-01, 5.2938825e-03],
       [9.2368059e-02, 9.0233803e-01, 5.2938825e-03],
       [9.8886624e-02, 8.9535761e-01, 5.7557393e-03]], dtype=float32)

In [44]:
pred_labels_dense = np.argmax(dense_pred, axis=1) -1; 
np.unique(pred_labels_dense, return_counts=True)

(array([-1,  0,  1]), array([ 1465, 10355,   871]))

In [45]:
from sklearn.metrics import balanced_accuracy_score, accuracy_score

In [46]:
print('CNN')
print('balanced: ', balanced_accuracy_score(pred_labels_cnn, labels_orig))
print('accuracy: ', accuracy_score(pred_labels_cnn, labels_orig))

CNN
balanced:  0.6443471522590293
accuracy:  0.790481443542668


In [47]:
print('DENSE')
print('balanced: ', balanced_accuracy_score(pred_labels_dense, labels_orig))
print('accuracy: ', accuracy_score(pred_labels_dense, labels_orig))

DENSE
balanced:  0.6492241012953273
accuracy:  0.7948940193838153
